In [1]:
import umap
import pickle
import joblib
import random
import string
import warnings
warnings.simplefilter('ignore')

import numpy as np
import holoviews as hv
import pandas as pd



hv.extension('bokeh')
data_dir = '/home/cdsw/airline-sentiment/data/'
model_dir = '/home/cdsw/airline-sentiment/model/'

ModuleNotFoundError: No module named 'umap'

In [2]:
with open(data_dir+'/frontend_data', 'rb') as f:
    data = pickle.load(f)

In [3]:
data.keys()

dict_keys(['prediction', 'embedding', 'tweet', 'airline'])

In [4]:
len(data['prediction'])

1731

In [5]:
data_df = pd.DataFrame(data)

In [6]:
data_df.head()

,prediction,embedding,tweet,airline
0,0.141585,"[[-0.9406724, 0.9982645, -0.99448645, 0.972652...","[@united, was, such, a, better, airline, when,...",[United]
1,0.149422,"[[-0.9352172, 0.9990309, -0.9944742, 0.9831363...","[@united, wi, -, fi, on, 737, -, 800, (, aircr...",[United]
2,0.128639,"[[-0.9330108, 0.9983444, -0.9919408, 0.9770202...","[@USAirways, Only, the, JFK, baggage, office, ...","[US, Airways]"
3,0.137313,"[[-0.9453927, 0.9987324, -0.99320173, 0.980086...","[@AmericanAir, is, there, something, wrong, wi...",[American]
4,0.135864,"[[-0.9473117, 0.9989401, -0.9929571, 0.9809232...","[@USAirways, So, when, was, I, supposed, to, c...","[US, Airways]"


In [7]:
#data_df.airline.value_counts()

In [8]:
np.shape(data['embedding'][0])

(1, 256)

In [9]:
X = np.vstack(data_df['embedding'].values)

np.shape(X)

(1731, 256)

In [10]:
%%time 

embedding = umap.UMAP(n_components=2,
                      n_neighbors=15,
                      min_dist=0.1,
                      metric='cosine').fit_transform(X)

/home/cdsw/.local/lib/python3.6/site-packages/umap/umap_.py:349: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fuzzy_simplicial_set" failed type inference due to: Untyped global name 'nearest_neighbors': cannot determine Numba type of <class 'function'>

File "../../.local/lib/python3.6/site-packages/umap/umap_.py", line 467:
def fuzzy_simplicial_set(
    <source elided>
    if knn_indices is None or knn_dists is None:
        knn_indices, knn_dists, _ = nearest_neighbors(
        ^

  @numba.jit()
/home/cdsw/.local/lib/python3.6/site-packages/numba/compiler.py:725: NumbaWarning: Function "fuzzy_simplicial_set" was compiled in object mode without forceobj=True.

File "../../.local/lib/python3.6/site-packages/umap/umap_.py", line 350:
@numba.jit()
def fuzzy_simplicial_set(
^

  self.func_ir.loc))
/home/cdsw/.local/lib/python3.6/site-packages/numba/compiler.py:734: NumbaDeprecationWarning: 
Fall-back from the nopython compilation pat

CPU times: user 14.4 s, sys: 184 ms, total: 14.6 s
Wall time: 14.6 s


In [11]:
data_df['umap_x'], data_df['umap_y'] = embedding[:, 0], embedding[:, 1]

In [12]:
%%opts Scatter [width=500 height=500] (color='prediction')
hv.Scatter(data_df, kdims=['umap_x', 'umap_y'], vdims=['prediction'])

:Scatter   [umap_x,umap_y]   (prediction)

In [13]:
#data_df['tweet'] = [''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
#                    for i in range(0, len(data_df))]

In [14]:
data_df.head()

,prediction,embedding,tweet,airline,umap_x,umap_y
0,0.141585,"[[-0.9406724, 0.9982645, -0.99448645, 0.972652...","[@united, was, such, a, better, airline, when,...",[United],-6.354976,3.010093
1,0.149422,"[[-0.9352172, 0.9990309, -0.9944742, 0.9831363...","[@united, wi, -, fi, on, 737, -, 800, (, aircr...",[United],-0.507132,-3.451891
2,0.128639,"[[-0.9330108, 0.9983444, -0.9919408, 0.9770202...","[@USAirways, Only, the, JFK, baggage, office, ...","[US, Airways]",1.276108,13.485870
3,0.137313,"[[-0.9453927, 0.9987324, -0.99320173, 0.980086...","[@AmericanAir, is, there, something, wrong, wi...",[American],-0.834772,12.863005
4,0.135864,"[[-0.9473117, 0.9989401, -0.9929571, 0.9809232...","[@USAirways, So, when, was, I, supposed, to, c...","[US, Airways]",-14.246212,0.219514


In [15]:
data_df['tweet'] = data_df['tweet'].apply(lambda t: " ".join(t))

In [16]:
joblib.dump(data_df, data_dir+'/umap_embedding.joblib', compress=True)

['/home/cdsw/airline-sentiment/data//umap_embedding.joblib']